In [1]:
# Converts PIPS data from the raw TOB32 to TOA5 format
# uses external system call to camp2ascii

In [2]:
import os
import sys
import subprocess
import glob
import re

In [3]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [atoi(c) for c in re.split(r'(\d+)', text)]

In [4]:
basedir = '/Users/dawson29/Dropbox/Projects/TriPIPS'
inputsubdir = 'incoming_111018_120918'
outputsubdir = 'converted_111018_120918'
inputdir = os.path.join(basedir, inputsubdir)
outputdir = os.path.join(basedir, outputsubdir)
if not os.path.exists(outputdir):
    os.makedirs(outputdir)
filelist = glob.glob(inputdir+'/*.dat')
filelist.sort(key=natural_keys)
print(filelist)

['/Users/dawson29/Dropbox/Projects/TriPIPS/incoming_111018_120918/Met_Location_OneHz560.dat', '/Users/dawson29/Dropbox/Projects/TriPIPS/incoming_111018_120918/Met_Location_OneHz561.dat', '/Users/dawson29/Dropbox/Projects/TriPIPS/incoming_111018_120918/Met_Location_OneHz562.dat', '/Users/dawson29/Dropbox/Projects/TriPIPS/incoming_111018_120918/Met_Location_OneHz563.dat', '/Users/dawson29/Dropbox/Projects/TriPIPS/incoming_111018_120918/Met_Location_OneHz564.dat', '/Users/dawson29/Dropbox/Projects/TriPIPS/incoming_111018_120918/Met_Location_OneHz565.dat', '/Users/dawson29/Dropbox/Projects/TriPIPS/incoming_111018_120918/Met_Location_OneHz566.dat', '/Users/dawson29/Dropbox/Projects/TriPIPS/incoming_111018_120918/Met_Location_OneHz567.dat', '/Users/dawson29/Dropbox/Projects/TriPIPS/incoming_111018_120918/Met_Location_OneHz568.dat', '/Users/dawson29/Dropbox/Projects/TriPIPS/incoming_111018_120918/Met_Location_OneHz569.dat', '/Users/dawson29/Dropbox/Projects/TriPIPS/incoming_111018_120918/Met_

In [5]:
# Convert each file one-by-one
for rawfile in filelist:
    outfile = 'TAO5_' + os.path.basename(rawfile)
    outpath = os.path.join(outputdir, outfile)
    if not os.path.exists(outpath):
        print("Processing {}".format(outfile))
        try:
            output = subprocess.run(['camp2ascii', rawfile], stdout=subprocess.PIPE, universal_newlines=True, errors='ignore')
            with open(outpath, 'w') as f:
                f.write(output.stdout)
        except:
            print("Cannot read file! Skipping!")
    else:
        print("Skipping {}".format(outfile))

Processing TAO5_Met_Location_OneHz560.dat
Processing TAO5_Met_Location_OneHz561.dat
Processing TAO5_Met_Location_OneHz562.dat
Processing TAO5_Met_Location_OneHz563.dat
Processing TAO5_Met_Location_OneHz564.dat
Processing TAO5_Met_Location_OneHz565.dat
Processing TAO5_Met_Location_OneHz566.dat
Processing TAO5_Met_Location_OneHz567.dat
Processing TAO5_Met_Location_OneHz568.dat
Processing TAO5_Met_Location_OneHz569.dat
Processing TAO5_Met_Location_OneHz570.dat
Processing TAO5_Met_Location_OneHz571.dat
Processing TAO5_Met_Location_OneHz572.dat
Processing TAO5_Met_Location_OneHz573.dat
Processing TAO5_Met_Location_OneHz574.dat
Processing TAO5_Met_Location_OneHz575.dat
Processing TAO5_Met_Location_OneHz576.dat
Processing TAO5_Met_Location_OneHz577.dat
Processing TAO5_Met_Location_OneHz578.dat
Processing TAO5_Met_Location_OneHz579.dat
Processing TAO5_Met_Location_OneHz580.dat
Processing TAO5_Met_Location_OneHz581.dat
Processing TAO5_Met_Location_OneHz582.dat
Processing TAO5_Met_Location_OneHz

Processing TAO5_Met_Location_OneHz760.dat
Processing TAO5_Met_Location_OneHz761.dat
Processing TAO5_Met_Location_OneHz762.dat
Processing TAO5_Met_Location_OneHz763.dat
Processing TAO5_Met_Location_OneHz764.dat
Processing TAO5_Met_Location_OneHz765.dat
Processing TAO5_Met_Location_OneHz766.dat
Processing TAO5_Met_Location_OneHz767.dat
Processing TAO5_Met_Location_OneHz768.dat
Processing TAO5_Met_Location_OneHz769.dat
Processing TAO5_Met_Location_OneHz770.dat
Processing TAO5_Met_Location_OneHz771.dat
Processing TAO5_Met_Location_OneHz772.dat
Processing TAO5_Met_Location_OneHz773.dat
Processing TAO5_Met_Location_OneHz774.dat
Processing TAO5_Met_Location_OneHz775.dat
Processing TAO5_Met_Location_OneHz776.dat
Processing TAO5_Met_Location_OneHz777.dat
Processing TAO5_Met_Location_OneHz778.dat
Processing TAO5_Met_Location_OneHz779.dat
Processing TAO5_Met_Location_OneHz780.dat
Processing TAO5_Met_Location_OneHz781.dat
Processing TAO5_Met_Location_OneHz782.dat
Processing TAO5_Met_Location_OneHz

Processing TAO5_Met_Location_OneHz960.dat
Processing TAO5_Met_Location_OneHz961.dat
Processing TAO5_Met_Location_OneHz962.dat
Processing TAO5_Met_Location_OneHz963.dat
Processing TAO5_Met_Location_OneHz964.dat
Processing TAO5_Met_Location_OneHz965.dat
Processing TAO5_Met_Location_OneHz966.dat
Processing TAO5_Met_Location_OneHz967.dat
Processing TAO5_Met_Location_OneHz968.dat
Processing TAO5_Met_Location_OneHz969.dat
Processing TAO5_Met_Location_OneHz970.dat
Processing TAO5_Met_Location_OneHz971.dat
Processing TAO5_Met_Location_OneHz972.dat
Processing TAO5_Met_Location_OneHz973.dat
Processing TAO5_Met_Location_OneHz974.dat
Processing TAO5_Met_Location_OneHz975.dat
Processing TAO5_Met_Location_OneHz976.dat
Processing TAO5_Met_Location_OneHz977.dat
Processing TAO5_Met_Location_OneHz978.dat
Processing TAO5_Met_Location_OneHz979.dat
Processing TAO5_Met_Location_OneHz980.dat
Processing TAO5_Met_Location_OneHz981.dat
Processing TAO5_Met_Location_OneHz982.dat
Processing TAO5_Met_Location_OneHz

Processing TAO5_Met_Location_OneHz1160.dat
Processing TAO5_Met_Location_OneHz1161.dat
Processing TAO5_Met_Location_OneHz1162.dat
Processing TAO5_Met_Location_OneHz1163.dat
Processing TAO5_Met_Location_OneHz1164.dat
Processing TAO5_Met_Location_OneHz1165.dat
Processing TAO5_Met_Location_OneHz1166.dat
Processing TAO5_Met_Location_OneHz1167.dat
Processing TAO5_Met_Location_OneHz1168.dat
Processing TAO5_Met_Location_OneHz1169.dat
Processing TAO5_Met_Location_OneHz1170.dat
Processing TAO5_Met_Location_OneHz1171.dat
Processing TAO5_Met_Location_OneHz1172.dat
Processing TAO5_Met_Location_OneHz1173.dat
Processing TAO5_Met_Location_OneHz1174.dat
Processing TAO5_Met_Location_OneHz1175.dat
Processing TAO5_Met_Location_OneHz1176.dat
Processing TAO5_Met_Location_OneHz1177.dat
Processing TAO5_Met_Location_OneHz1178.dat
Processing TAO5_Met_Location_OneHz1179.dat
Processing TAO5_Met_Location_OneHz1180.dat
Processing TAO5_Met_Location_OneHz1181.dat
Processing TAO5_Met_Location_OneHz1182.dat
Processing 

Processing TAO5_Met_Location_OneHz1360.dat
Processing TAO5_Met_Location_OneHz1361.dat
Processing TAO5_Met_Location_OneHz1362.dat
Processing TAO5_Met_Location_OneHz1363.dat
Processing TAO5_Met_Location_OneHz1364.dat
Processing TAO5_Met_Location_OneHz1365.dat
Processing TAO5_Met_Location_OneHz1366.dat
Processing TAO5_Met_Location_OneHz1367.dat
Processing TAO5_Met_Location_OneHz1368.dat
Processing TAO5_Met_Location_OneHz1369.dat
Processing TAO5_Met_Location_OneHz1370.dat
Processing TAO5_Met_Location_OneHz1371.dat
Processing TAO5_Met_Location_OneHz1372.dat
Processing TAO5_Met_Location_OneHz1373.dat
Processing TAO5_Met_Location_OneHz1374.dat
Processing TAO5_Met_Location_OneHz1375.dat
Processing TAO5_Met_Location_OneHz1376.dat
Processing TAO5_Met_Location_OneHz1377.dat
Processing TAO5_Met_Location_OneHz1378.dat
Processing TAO5_Met_Location_OneHz1379.dat
Processing TAO5_Met_Location_OneHz1380.dat
Processing TAO5_Met_Location_OneHz1381.dat
Processing TAO5_Met_Location_OneHz1382.dat
Processing 

...

Processing TAO5_Parsivel_TenHz7787.dat
Processing TAO5_Parsivel_TenHz7788.dat
Processing TAO5_Parsivel_TenHz7789.dat
Processing TAO5_Parsivel_TenHz7790.dat
Processing TAO5_Parsivel_TenHz7791.dat
Processing TAO5_Parsivel_TenHz7792.dat
Processing TAO5_Parsivel_TenHz7793.dat
Processing TAO5_Parsivel_TenHz7794.dat
Processing TAO5_Parsivel_TenHz7795.dat
Processing TAO5_Parsivel_TenHz7796.dat
Processing TAO5_Parsivel_TenHz7797.dat
Processing TAO5_Parsivel_TenHz7798.dat
Processing TAO5_Parsivel_TenHz7799.dat
Processing TAO5_Parsivel_TenHz7800.dat
Processing TAO5_Parsivel_TenHz7801.dat
Processing TAO5_Parsivel_TenHz7802.dat
Processing TAO5_Parsivel_TenHz7803.dat
Processing TAO5_Parsivel_TenHz7804.dat
Processing TAO5_Parsivel_TenHz7805.dat
Processing TAO5_Parsivel_TenHz7806.dat
Processing TAO5_Parsivel_TenHz7807.dat
Processing TAO5_Parsivel_TenHz7808.dat
Processing TAO5_Parsivel_TenHz7809.dat
Processing TAO5_Parsivel_TenHz7810.dat
Processing TAO5_Parsivel_TenHz7811.dat
Processing TAO5_Parsivel_

Processing TAO5_Parsivel_TenHz8002.dat
Processing TAO5_Parsivel_TenHz8003.dat
Processing TAO5_Parsivel_TenHz8004.dat
Processing TAO5_Parsivel_TenHz8005.dat
Processing TAO5_Parsivel_TenHz8006.dat
Processing TAO5_Parsivel_TenHz8007.dat
Processing TAO5_Parsivel_TenHz8008.dat
Processing TAO5_Parsivel_TenHz8009.dat
Processing TAO5_Parsivel_TenHz8010.dat
Processing TAO5_Parsivel_TenHz8011.dat
Processing TAO5_Parsivel_TenHz8012.dat
Processing TAO5_Parsivel_TenHz8013.dat
Processing TAO5_Parsivel_TenHz8014.dat
Processing TAO5_Parsivel_TenHz8015.dat
Processing TAO5_Parsivel_TenHz8016.dat
Processing TAO5_Parsivel_TenHz8017.dat
Processing TAO5_Parsivel_TenHz8018.dat
Processing TAO5_Parsivel_TenHz8019.dat
Processing TAO5_Parsivel_TenHz8020.dat
Processing TAO5_Parsivel_TenHz8021.dat
Processing TAO5_Parsivel_TenHz8022.dat
Processing TAO5_Parsivel_TenHz8023.dat
Processing TAO5_Parsivel_TenHz8024.dat
Processing TAO5_Parsivel_TenHz8025.dat
Processing TAO5_Parsivel_TenHz8026.dat
Processing TAO5_Parsivel_

Processing TAO5_Parsivel_TenHz8216.dat
Processing TAO5_Parsivel_TenHz8217.dat
Processing TAO5_Parsivel_TenHz8218.dat
Processing TAO5_Parsivel_TenHz8219.dat
Processing TAO5_Parsivel_TenHz8220.dat
Processing TAO5_Parsivel_TenHz8221.dat
Processing TAO5_Parsivel_TenHz8222.dat
Processing TAO5_Parsivel_TenHz8223.dat
Processing TAO5_Parsivel_TenHz8224.dat
Processing TAO5_Parsivel_TenHz8225.dat
Processing TAO5_Parsivel_TenHz8226.dat
Processing TAO5_Parsivel_TenHz8227.dat
Processing TAO5_Parsivel_TenHz8228.dat
Processing TAO5_Parsivel_TenHz8229.dat
Processing TAO5_Parsivel_TenHz8230.dat
Processing TAO5_Parsivel_TenHz8231.dat
Processing TAO5_Parsivel_TenHz8232.dat
Processing TAO5_Parsivel_TenHz8233.dat
Processing TAO5_Parsivel_TenHz8234.dat
Processing TAO5_Parsivel_TenHz8235.dat
Processing TAO5_Parsivel_TenHz8236.dat
Processing TAO5_Parsivel_TenHz8237.dat
Processing TAO5_Parsivel_TenHz8238.dat
Processing TAO5_Parsivel_TenHz8239.dat
Processing TAO5_Parsivel_TenHz8240.dat
Processing TAO5_Parsivel_

Processing TAO5_Parsivel_TenHz8429.dat
Processing TAO5_Parsivel_TenHz8430.dat
Processing TAO5_Parsivel_TenHz8431.dat
Processing TAO5_Parsivel_TenHz8432.dat
Processing TAO5_Parsivel_TenHz8433.dat
Processing TAO5_Parsivel_TenHz8434.dat
Processing TAO5_Parsivel_TenHz8435.dat
Processing TAO5_Parsivel_TenHz8436.dat
Processing TAO5_Parsivel_TenHz8437.dat
Processing TAO5_Parsivel_TenHz8438.dat
Processing TAO5_Parsivel_TenHz8439.dat
Processing TAO5_Parsivel_TenHz8440.dat
Processing TAO5_Parsivel_TenHz8441.dat
Processing TAO5_Parsivel_TenHz8442.dat
Processing TAO5_Parsivel_TenHz8443.dat
Processing TAO5_Parsivel_TenHz8444.dat
Processing TAO5_Parsivel_TenHz8445.dat
Processing TAO5_Parsivel_TenHz8446.dat
Processing TAO5_Parsivel_TenHz8447.dat
Processing TAO5_Parsivel_TenHz8448.dat
Processing TAO5_Parsivel_TenHz8449.dat
Processing TAO5_Parsivel_TenHz8450.dat
Processing TAO5_Parsivel_TenHz8451.dat
Processing TAO5_Parsivel_TenHz8452.dat
Processing TAO5_Parsivel_TenHz8453.dat
Processing TAO5_Parsivel_

Processing TAO5_Parsivel_TenHz8647.dat
Processing TAO5_Parsivel_TenHz8648.dat
Processing TAO5_Parsivel_TenHz8649.dat
Processing TAO5_Parsivel_TenHz8650.dat
Processing TAO5_Parsivel_TenHz8651.dat
Processing TAO5_Parsivel_TenHz8652.dat
Processing TAO5_Parsivel_TenHz8653.dat
Processing TAO5_Parsivel_TenHz8654.dat
Processing TAO5_Parsivel_TenHz8655.dat
Processing TAO5_Parsivel_TenHz8656.dat
Processing TAO5_Parsivel_TenHz8657.dat
Processing TAO5_Parsivel_TenHz8658.dat
Processing TAO5_Parsivel_TenHz8659.dat
Processing TAO5_Parsivel_TenHz8660.dat
Processing TAO5_Parsivel_TenHz8661.dat
Processing TAO5_Parsivel_TenHz8662.dat
Processing TAO5_Parsivel_TenHz8663.dat
Processing TAO5_Parsivel_TenHz8664.dat
Processing TAO5_Parsivel_TenHz8665.dat
Processing TAO5_Parsivel_TenHz8666.dat
Processing TAO5_Parsivel_TenHz8667.dat
Processing TAO5_Parsivel_TenHz8668.dat
Processing TAO5_Parsivel_TenHz8669.dat
Processing TAO5_Parsivel_TenHz8670.dat
Processing TAO5_Parsivel_TenHz8671.dat
Processing TAO5_Parsivel_

In [6]:
# Fix converted one-sec files that have bad data at the end

filelist = glob.glob(outputdir+'/*Met_Location*.dat')

for converted_file in filelist:
    badfile = False
    newlines = []
    with open(converted_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            tokens = line.strip().split(',')
            if "1944" in tokens[0]:
                if not badfile:
                    print("Found bad file {}! Fixing!".format(converted_file))
                badfile = True
                break
            else:
                newlines.append(line)
    if badfile:
        new_file = converted_file + '_fixed'
        with open(new_file, 'w') as f:
            f.writelines(newlines)
    
                

Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz7533.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz8600.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz5324.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz5442.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz4984.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz7255.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz8166.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz7241.dat! Fixing!
Found bad file /Users/dawson29/D

Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz5913.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz8589.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz7862.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz7876.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz5907.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz7447.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz6981.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz6759.dat! Fixing!
Found bad file /Users/dawson29/D

Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz5721.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz5735.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz7122.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz7644.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz8577.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz5053.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz7903.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz5872.dat! Fixing!
Found bad file /Users/dawson29/D

Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz6612.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz6821.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz4788.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz5496.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz4950.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz7281.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz7295.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz5482.dat! Fixing!
Found bad file /Users/dawson29/D

Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz8039.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz6000.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz4617.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz5509.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz7478.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz6766.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz7493.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz6955.dat! Fixing!
Found bad file /Users/dawson29/D

...

Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz7313.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz8020.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz5504.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz5262.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz7475.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz8746.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz6743.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz8008.dat! Fixing!
Found bad file /Users/dawson29/D

Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz6838.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz7515.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz8626.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz5302.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz5464.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz7273.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz8140.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz6179.dat! Fixing!
Found bad file /Users/dawson29/D

Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz5738.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz4426.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz6231.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz7891.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz6557.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz7649.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz8552.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz7661.dat! Fixing!
Found bad file /Users/dawson29/D

Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz4396.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz5088.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz6581.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz7847.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz5936.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz5705.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz7112.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz7674.dat! Fixing!
Found bad file /Users/dawson29/D

Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz8619.dat! Fixing!
Found bad file /Users/dawson29/Dropbox/Projects/TriPIPS/converted_111018_120918/TAO5_Met_Location_OneHz6634.dat! Fixing!
